In [286]:
import random
from typing import List, Iterator, Callable, Tuple

class Card: 
    def __init__(self, month : int, is_gwang : bool = False, is_yul : bool = False) :
        self.month = month
        self.is_gwang = is_gwang
        self.is_yul = is_yul
        
    def show(self) -> str:
        if self.is_gwang:
            return f"{self.month}월 Gwang"
        if self.is_yul:
            return f"{self.month}월 Yul"
        return f"{self.month}월"

class Deck:
    def __init__(self):
        self.init_deck()

    def __iter__(self) -> Iterator[Card]:
        return iter(self.deck)
        
    def __len__(self) -> int:
        return len(self.deck)
        
    def show(self) -> List[str]:
        return [i.show() for i in self.deck]

    def init_deck(self) -> List[Card]:
        self.deck : List[Card] = [
            Card(1, is_gwang=True), Card(1), Card(1), Card(1), 
            Card(2, is_yul = True), Card(2), Card(2), Card(2),
            Card(3, is_gwang=True), Card(3), Card(3), Card(3),
            Card(4, is_yul = True), Card(4), Card(4), Card(4),
            Card(5, is_yul = True), Card(5), Card(5), Card(5),
            Card(6, is_yul=True), Card(6), Card(6), Card(6),
            Card(7, is_yul=True), Card(7), Card(7), Card(7),
            Card(8, is_gwang=True), Card(8, is_yul = True), Card(8), Card(8),
            Card(9, is_yul=True), Card(9), Card(9), Card(9),
            Card(10, is_yul=True), Card(10), Card(10), Card(10)
        ]
        return self.deck

    def reset(self):
        self.deck = self.init_deck()

    def shuffle(self) -> None:
        random.shuffle(self.deck)

    def draw(self, n : int = 1) -> List[Card]:
        # 카드중 하나를 비복원 추출
        if n > len(self.deck):
            raise ValueError("덱에 카드가 부족합니다.")
        drawn = self.deck[:n]
        self.deck = self.deck[n:]
        return drawn

class Player:
    def __init__(self, name: str):
        self.name: str = name
        self.hand: List[Card] = []  # 두 장의 카드
        self.is_ready: bool = False
        self.result = None
        self.score = None  # 족보 점수 (비교용)

    def __repr__(self) -> str:
        return f"<Player {self.name}, Hand: {self.hand}>"

    def receive_card(self, cards: Card) -> None:
        # 카드 2장을 받는다.
        if len(self.hand) == 2 :
            self.hand: List[Card] = []
            self.hand.extend(cards)
        else :
            self.hand.extend(cards)

    def reset(self) -> None:
        # 새 라운드를 위한 초기화
        self.hand.clear()
        self.result = ('', 0)
        self.is_ready = False
        self.score = None

    

class GameRoom:
    def __init__(self, room_name: str,  jokbo : list, max_players: int = 4):
        self.room_name: str = room_name
        self.max_players: int = max_players
        self.players: List[Player] = []
        self.deck = Deck()
        self.started: bool = False
        self.jokbo = jokbo

    def add_player(self, player: Player) -> None:
        if len(self.players) >= self.max_players:
            raise Exception("플레이어 수 초과")
        self.players.append(player)

    def start_game(self, is_regame = False, is_test = False) -> None:
        self.deck.reset()
        self.deck.shuffle()
        if is_test :
            pass  
        elif is_regame:
            for i in range(2): # 플레이어 카드 분배 (한장씩 두바퀴)
                for player in self.players:
                    player.receive_card(self.deck.draw()) 
        else :
            if len(self.players) < 4:
                raise Exception("플레이어가 부족합니다.")
            for i in range(2): # 플레이어 카드 분배 (한장씩 두바퀴)
                for player in self.players:
                    player.receive_card(self.deck.draw())
        self.started = True     
        self.calculate()   
        return self.show_all_result(), self.get_winner()
                
    def calculate(self) -> None:   
        for player in self.players:
            player.result = None 
            player.hand.sort(key=lambda card: card.month)      
            # 땡, 광땡, 중간족보, 특수족보
            for idx, hand in enumerate(self.jokbo) :
                if hand['condition'](player.hand[0], player.hand[1]):
                    if hand['name'] == "땡" :
                        if (player.hand[0].month + player.hand[0].month) == 20 : # 10땡은 장땡으로 변경
                            player.result = ["장땡", player.hand[0].month + hand['score']]
                            break
                        else :
                            player.result = ["땡", player.hand[0].month + hand['score']]
                            break
                    else :
                        player.result = [hand['name'], hand['score']]
                        break
                        
            # 족보에 속하지 않는 경우 : n끗
            if not player.result : 
                player.result = ["끗", (player.hand[0].month + player.hand[1].month) % 10]
                
            # 어떤 족보에도 속하지 않는경우
            elif not player.result :
                raise Exception("점수계산에 오류가 발생했습니다")

    def get_winner(self):
        player_result = [p.result[0] for p in self.players]
        sorted_players = sorted( self.players, key=lambda p: p.result[1], reverse=True)
        if "암행어사" in player_result and set(player_result) & set(["일삼광땡", "일팔광땡"]):
            self.players[player_result.index("암행어사")].result[1] = 1200
                    
        if "땡잡이" in player_result and "땡" in player_result :
            self.players[player_result.index("땡잡이")].result[1] = 110
        
        if "멍텅구리 구사" in player_result and sorted_players[0].result[1] <= 109 :
            return self.start_game(is_regame=True)
            
        if "구사" in player_result and sorted_players[0].result[1] <= 15 :
            return self.start_game(is_regame=True)

        # 특수 규칙 계산 이후 다시 정렬
        sorted_players = sorted(
            self.players,
            key=lambda p: p.result[1],
            reverse=True
        )
        
        # 동점 발생시 게임 재시작
        winner = sorted_players[0]
        regame_player = []
        for p in sorted_players:
            if winner.result[1] == p.result[1] :
                regame_player.append(p)

        if len(regame_player) > 1 :
            self.players = regame_player
            return self.start_game(is_regame=True)
   
        else : 
            return f"{winner.name} 우승"

    def reset_all_game(self) -> None:
        self.deck.reset()
        for player in self.players:
            player.reset()
        self.started = False
    
    def show_all_hands(self):
        return {player.name: [card.show() for card in player.hand] for player in self.players}

    def show_all_result(self):
        return {player.name: player.result for player in self.players}      

In [310]:
jokbo = [
    {"name": "일삼광땡", "condition": lambda c1, c2: [c1.month, c2.month] == [1,3] and c1.is_gwang and c2.is_gwang, "score" : 1000},
    {"name": "일팔광땡", "condition": lambda c1, c2: [c1.month, c2.month] == [1,8] and c1.is_gwang and c2.is_gwang, "score" : 1100},
    {"name": "삼팔광땡", "condition": lambda c1, c2: [c1.month, c2.month] == [3,8] and c1.is_gwang and c2.is_gwang, "score" : 1300},
    
    {"name": "암행어사", "condition": lambda c1, c2: [c1.month, c2.month] == [4,7] and (c1.is_yul and c2.is_yul), "score" : 1},
    {"name": "땡잡이", "condition": lambda c1, c2: [c1.month, c2.month] == [3,7] and (c1.is_gwang and c2.is_yul), "score" : 0}, # +110
    {"name": "멍텅구리 구사", "condition": lambda c1, c2: [c1.month, c2.month] == [4, 9] and (c1.is_yul and c2.is_yul), "score" : 3}, # 구땡이하 재경기 +110
    {"name": "구사", "condition": lambda c1, c2: [c1.month, c2.month] == [4, 9] and not(c1.is_yul and c2.is_yul), "score" : 3}, # 알리 이하 재시작
    
    {"name": "갑오", "condition": lambda c1, c2: (c1.month + c2.month) % 10 == 9, "score" : 9},
    {"name": "세륙", "condition": lambda c1, c2: [c1.month, c2.month] == [4, 6], "score" : 10},
    {"name": "장사", "condition": lambda c1, c2: [c1.month, c2.month] == [4, 10], "score" : 11},
    {"name": "장삥", "condition": lambda c1, c2: [c1.month, c2.month] == [1, 10], "score" : 12},
    {"name": "구삥", "condition": lambda c1, c2: [c1.month, c2.month] == [1, 9], "score" : 13},
    {"name": "독사", "condition": lambda c1, c2: [c1.month, c2.month] == [1, 4], "score" : 14},
    {"name": "알리", "condition": lambda c1, c2: [c1.month, c2.month] == [1, 2], "score" : 15},
    {"name": "망통", "condition": lambda c1, c2: (c1.month + c2.month) % 10 == 0, "score" : 0},
    
    {"name": "땡", "condition": lambda c1, c2: c1.month == c2.month, "score" : 100}, # 장땡 구현하기
]


In [288]:
# TC01

deck = Deck()
game = GameRoom("방1", jokbo=jokbo)

# 플레이어 추가
player1 = Player("p1")
player1.receive_card([Card(4, is_yul=True)])
player1.receive_card([Card(7, is_yul=True)])
player2 = Player("p2")
player2.receive_card([Card(1, is_gwang=True)])
player2.receive_card([Card(3, is_gwang=True)])
player3 = Player("p3")
player3.receive_card([Card(3)])
player3.receive_card([Card(7)])
player4 = Player("p4")
player4.receive_card([Card(3)])
player4.receive_card([Card(7)])

game.add_player(player1)
game.add_player(player2)
game.add_player(player3)
game.add_player(player4)

game.start_game(is_test=True)

({'p1': ['암행어사', 1200],
  'p2': ['일삼광땡', 1000],
  'p3': ['망통', 0],
  'p4': ['망통', 0]},
 'p1 우승')

In [289]:
# TC02

deck = Deck()
game = GameRoom("방1", jokbo=jokbo)

# 플레이어 추가
player1 = Player("p1")
player1.receive_card([Card(4, is_yul=True)])
player1.receive_card([Card(7, is_yul=True)])
player2 = Player("p2")
player2.receive_card([Card(3, is_gwang=True)])
player2.receive_card([Card(8, is_gwang=True)])
player3 = Player("p3")
player3.receive_card([Card(3)])
player3.receive_card([Card(7)])
player4 = Player("p4")
player4.receive_card([Card(3)])
player4.receive_card([Card(7)])

game.add_player(player1)
game.add_player(player2)
game.add_player(player3)
game.add_player(player4)

game.start_game(is_test=True)

({'p1': ['암행어사', 1], 'p2': ['삼팔광땡', 1300], 'p3': ['망통', 0], 'p4': ['망통', 0]},
 'p2 우승')

In [290]:
# TC03

deck = Deck()
game = GameRoom("방1", jokbo=jokbo)

# 플레이어 추가
player1 = Player("p1")
player1.receive_card([Card(4, is_yul=True)])
player1.receive_card([Card(7, is_yul=True)])
player2 = Player("p2")
player2.receive_card([Card(3, is_gwang=True)])
player2.receive_card([Card(7, is_yul=True)])
player3 = Player("p3")
player3.receive_card([Card(9)])
player3.receive_card([Card(9)])
player4 = Player("p4")
player4.receive_card([Card(3)])
player4.receive_card([Card(7)])

game.add_player(player1)
game.add_player(player2)
game.add_player(player3)
game.add_player(player4)

game.start_game(is_test=True)

({'p1': ['암행어사', 1], 'p2': ['땡잡이', 110], 'p3': ['땡', 109], 'p4': ['망통', 0]},
 'p2 우승')

In [291]:
# TC04

deck = Deck()
game = GameRoom("방1", jokbo=jokbo)

# 플레이어 추가
player1 = Player("p1")
player1.receive_card([Card(4, is_yul=True)])
player1.receive_card([Card(7, is_yul=True)])
player2 = Player("p2")
player2.receive_card([Card(3, is_gwang=True)])
player2.receive_card([Card(7, is_yul=True)])
player3 = Player("p3")
player3.receive_card([Card(3)])
player3.receive_card([Card(7)])
player4 = Player("p4")
player4.receive_card([Card(3)])
player4.receive_card([Card(7)])

game.add_player(player1)
game.add_player(player2)
game.add_player(player3)
game.add_player(player4)

game.start_game(is_test=True)

({'p1': ['암행어사', 1], 'p2': ['땡잡이', 0], 'p3': ['망통', 0], 'p4': ['망통', 0]},
 'p1 우승')

In [292]:
# TC05

deck = Deck()
game = GameRoom("방1", jokbo=jokbo)

# 플레이어 추가
player1 = Player("p1")
player1.receive_card([Card(4, is_yul=True)])
player1.receive_card([Card(7, is_yul=True)])
player2 = Player("p2")
player2.receive_card([Card(3, is_gwang=True)])
player2.receive_card([Card(7, is_yul=True)])
player3 = Player("p3")
player3.receive_card([Card(4, is_yul=True)])
player3.receive_card([Card(9, is_yul=True)])
player4 = Player("p4")
player4.receive_card([Card(1, is_gwang=True)])
player4.receive_card([Card(8, is_gwang=True)])

game.add_player(player1)
game.add_player(player2)
game.add_player(player3)
game.add_player(player4)

game.start_game(is_test=True)

({'p1': ['암행어사', 1200],
  'p2': ['땡잡이', 0],
  'p3': ['멍텅구리 구사', 3],
  'p4': ['일팔광땡', 1100]},
 'p1 우승')

In [294]:
# TC06

deck = Deck()
game = GameRoom("방1", jokbo=jokbo)

# 플레이어 추가
player1 = Player("p1")
player1.receive_card([Card(4, is_yul=True)])
player1.receive_card([Card(7, is_yul=True)])
player2 = Player("p2")
player2.receive_card([Card(3, is_gwang=True)])
player2.receive_card([Card(7, is_yul=True)])
player3 = Player("p3")
player3.receive_card([Card(4, is_yul=True)])
player3.receive_card([Card(9, is_yul=True)])
player4 = Player("p4")
player4.receive_card([Card(9)])
player4.receive_card([Card(9)])

game.add_player(player1)
game.add_player(player2)
game.add_player(player3)
game.add_player(player4)

game.start_game(is_test=True)

({'p1': ['암행어사', 1],
  'p2': ['땡잡이', 110],
  'p3': ['멍텅구리 구사', 3],
  'p4': ['땡', 109]},
 ({'p1': ['끗', 4], 'p2': ['독사', 14], 'p3': ['끗', 3], 'p4': ['땡', 103]},
  'p4 우승'))

In [295]:
# TC07

deck = Deck()
game = GameRoom("방1", jokbo=jokbo)

# 플레이어 추가
player1 = Player("p1")
player1.receive_card([Card(4, is_yul=True)])
player1.receive_card([Card(7, is_yul=True)])
player2 = Player("p2")
player2.receive_card([Card(3, is_gwang=True)])
player2.receive_card([Card(7, is_yul=True)])
player3 = Player("p3")
player3.receive_card([Card(4, is_yul=True)])
player3.receive_card([Card(9, is_yul=True)])
player4 = Player("p4")
player4.receive_card([Card(1)])
player4.receive_card([Card(2)])

game.add_player(player1)
game.add_player(player2)
game.add_player(player3)
game.add_player(player4)

game.start_game(is_test=True)

({'p1': ['암행어사', 1], 'p2': ['땡잡이', 0], 'p3': ['멍텅구리 구사', 3], 'p4': ['알리', 15]},
 ({'p1': ['땡', 109], 'p2': ['끗', 4], 'p3': ['갑오', 9], 'p4': ['끗', 2]},
  'p1 우승'))

In [296]:
# TC08

deck = Deck()
game = GameRoom("방1", jokbo=jokbo)

# 플레이어 추가
player1 = Player("p1")
player1.receive_card([Card(4, is_yul=True)])
player1.receive_card([Card(7, is_yul=True)])
player2 = Player("p2")
player2.receive_card([Card(4, is_yul=True)])
player2.receive_card([Card(9, is_yul=True)])
player3 = Player("p3")
player3.receive_card([Card(1, is_gwang=True)])
player3.receive_card([Card(8, is_gwang=True)])
player4 = Player("p4")
player4.receive_card([Card(1)])
player4.receive_card([Card(2)])

game.add_player(player1)
game.add_player(player2)
game.add_player(player3)
game.add_player(player4)

game.start_game(is_test=True)

({'p1': ['암행어사', 1200],
  'p2': ['멍텅구리 구사', 3],
  'p3': ['일팔광땡', 1100],
  'p4': ['알리', 15]},
 'p1 우승')

In [297]:
# TC09

deck = Deck()
game = GameRoom("방1", jokbo=jokbo)

# 플레이어 추가
player1 = Player("p1")
player1.receive_card([Card(4, is_yul=True)])
player1.receive_card([Card(7, is_yul=True)])
player2 = Player("p2")
player2.receive_card([Card(4, is_yul=True)])
player2.receive_card([Card(9, is_yul=True)])
player3 = Player("p3")
player3.receive_card([Card(9)])
player3.receive_card([Card(9)])
player4 = Player("p4")
player4.receive_card([Card(1)])
player4.receive_card([Card(2)])

game.add_player(player1)
game.add_player(player2)
game.add_player(player3)
game.add_player(player4)

game.start_game(is_test=True)

({'p1': ['암행어사', 1], 'p2': ['멍텅구리 구사', 3], 'p3': ['땡', 109], 'p4': ['알리', 15]},
 ({'p1': ['끗', 8], 'p2': ['끗', 7], 'p3': ['끗', 8], 'p4': ['갑오', 9]}, 'p4 우승'))

In [298]:
# TC10

deck = Deck()
game = GameRoom("방1", jokbo=jokbo)

# 플레이어 추가
player1 = Player("p1")
player1.receive_card([Card(4, is_yul=True)])
player1.receive_card([Card(7, is_yul=True)])
player2 = Player("p2")
player2.receive_card([Card(4, is_yul=True)])
player2.receive_card([Card(9, is_yul=True)])
player3 = Player("p3")
player3.receive_card([Card(9)])
player3.receive_card([Card(4)])
player4 = Player("p4")
player4.receive_card([Card(1, is_gwang=True)])
player4.receive_card([Card(8, is_gwang=True)])

game.add_player(player1)
game.add_player(player2)
game.add_player(player3)
game.add_player(player4)

game.start_game(is_test=True)

({'p1': ['암행어사', 1200],
  'p2': ['멍텅구리 구사', 3],
  'p3': ['구사', 3],
  'p4': ['일팔광땡', 1100]},
 'p1 우승')

In [301]:
# TC11

deck = Deck()
game = GameRoom("방1", jokbo=jokbo)

# 플레이어 추가
player1 = Player("p1")
player1.receive_card([Card(4, is_yul=True)])
player1.receive_card([Card(7, is_yul=True)])
player2 = Player("p2")
player2.receive_card([Card(4, is_yul=True)])
player2.receive_card([Card(9, is_yul=True)])
player3 = Player("p3")
player3.receive_card([Card(9)])
player3.receive_card([Card(4)])
player4 = Player("p4")
player4.receive_card([Card(9)])
player4.receive_card([Card(9)])

game.add_player(player1)
game.add_player(player2)
game.add_player(player3)
game.add_player(player4)

game.start_game(is_test=True)

({'p1': ['암행어사', 1], 'p2': ['멍텅구리 구사', 3], 'p3': ['구사', 3], 'p4': ['땡', 109]},
 ({'p1': ['끗', 2], 'p2': ['갑오', 9], 'p3': ['끗', 6], 'p4': ['끗', 2]}, 'p2 우승'))

In [304]:
# TC12

deck = Deck()
game = GameRoom("방1", jokbo=jokbo)

# 플레이어 추가
player1 = Player("p1")
player1.receive_card([Card(4, is_yul=True)])
player1.receive_card([Card(7, is_yul=True)])
player2 = Player("p2")
player2.receive_card([Card(4, is_yul=True)])
player2.receive_card([Card(9, is_yul=True)])
player3 = Player("p3")
player3.receive_card([Card(9)])
player3.receive_card([Card(4)])
player4 = Player("p4")
player4.receive_card([Card(1)])
player4.receive_card([Card(2)])

game.add_player(player1)
game.add_player(player2)
game.add_player(player3)
game.add_player(player4)

game.start_game(is_test=True)

({'p1': ['암행어사', 1], 'p2': ['멍텅구리 구사', 3], 'p3': ['구사', 3], 'p4': ['알리', 15]},
 ({'p1': ['망통', 0], 'p2': ['끗', 2], 'p3': ['끗', 4], 'p4': ['끗', 1]}, 'p3 우승'))

In [305]:
# TC13

deck = Deck()
game = GameRoom("방1", jokbo=jokbo)

# 플레이어 추가
player1 = Player("p1")
player1.receive_card([Card(4, is_yul=True)])
player1.receive_card([Card(7, is_yul=True)])
player2 = Player("p2")
player2.receive_card([Card(4)])
player2.receive_card([Card(9)])
player3 = Player("p3")
player3.receive_card([Card(1, is_gwang=True)])
player3.receive_card([Card(8, is_gwang=True)])
player4 = Player("p4")
player4.receive_card([Card(1)])
player4.receive_card([Card(2)])

game.add_player(player1)
game.add_player(player2)
game.add_player(player3)
game.add_player(player4)

game.start_game(is_test=True)

({'p1': ['암행어사', 1200],
  'p2': ['구사', 3],
  'p3': ['일팔광땡', 1100],
  'p4': ['알리', 15]},
 'p1 우승')

In [306]:
# TC14

deck = Deck()
game = GameRoom("방1", jokbo=jokbo)

# 플레이어 추가
player1 = Player("p1")
player1.receive_card([Card(4, is_yul=True)])
player1.receive_card([Card(7, is_yul=True)])
player2 = Player("p2")
player2.receive_card([Card(4)])
player2.receive_card([Card(9)])
player3 = Player("p3")
player3.receive_card([Card(1)])
player3.receive_card([Card(2)])
player4 = Player("p4")
player4.receive_card([Card(1)])
player4.receive_card([Card(2)])

game.add_player(player1)
game.add_player(player2)
game.add_player(player3)
game.add_player(player4)

game.start_game(is_test=True)

({'p1': ['암행어사', 1], 'p2': ['구사', 3], 'p3': ['알리', 15], 'p4': ['알리', 15]},
 ({'p1': ['끗', 6], 'p2': ['갑오', 9], 'p3': ['알리', 15], 'p4': ['갑오', 9]},
  'p3 우승'))

In [307]:
# TC15

deck = Deck()
game = GameRoom("방1", jokbo=jokbo)

# 플레이어 추가
player1 = Player("p1")
player1.receive_card([Card(3, is_gwang=True)])
player1.receive_card([Card(7, is_yul=True)])
player2 = Player("p2")
player2.receive_card([Card(9)])
player2.receive_card([Card(9)])
player3 = Player("p3")
player3.receive_card([Card(1)])
player3.receive_card([Card(2)])
player4 = Player("p4")
player4.receive_card([Card(1)])
player4.receive_card([Card(2)])

game.add_player(player1)
game.add_player(player2)
game.add_player(player3)
game.add_player(player4)

game.start_game(is_test=True)

({'p1': ['땡잡이', 110], 'p2': ['땡', 109], 'p3': ['알리', 15], 'p4': ['알리', 15]},
 'p1 우승')

In [311]:
# TC16

deck = Deck()
game = GameRoom("방1", jokbo=jokbo)

# 플레이어 추가
player1 = Player("p1")
player1.receive_card([Card(3, is_gwang=True)])
player1.receive_card([Card(7, is_yul=True)])
player2 = Player("p2")
player2.receive_card([Card(1)])
player2.receive_card([Card(2)])
player3 = Player("p3")
player3.receive_card([Card(1)])
player3.receive_card([Card(4)])
player4 = Player("p4")
player4.receive_card([Card(1)])
player4.receive_card([Card(9)])

game.add_player(player1)
game.add_player(player2)
game.add_player(player3)
game.add_player(player4)

game.start_game(is_test=True)

({'p1': ['땡잡이', 0], 'p2': ['알리', 15], 'p3': ['독사', 14], 'p4': ['구삥', 13]},
 'p2 우승')

In [312]:
# TC17

deck = Deck()
game = GameRoom("방1", jokbo=jokbo)

# 플레이어 추가
player1 = Player("p1")
player1.receive_card([Card(3, is_gwang=True)])
player1.receive_card([Card(7, is_yul=True)])
player2 = Player("p2")
player2.receive_card([Card(4, is_yul=True)])
player2.receive_card([Card(9, is_yul=True)])
player3 = Player("p3")
player3.receive_card([Card(9)])
player3.receive_card([Card(9)])
player4 = Player("p4")
player4.receive_card([Card(1)])
player4.receive_card([Card(2)])

game.add_player(player1)
game.add_player(player2)
game.add_player(player3)
game.add_player(player4)

game.start_game(is_test=True)

({'p1': ['땡잡이', 110],
  'p2': ['멍텅구리 구사', 3],
  'p3': ['땡', 109],
  'p4': ['알리', 15]},
 ({'p1': ['독사', 14], 'p2': ['끗', 2], 'p3': ['끗', 6], 'p4': ['끗', 5]}, 'p1 우승'))

In [313]:
# TC18

deck = Deck()
game = GameRoom("방1", jokbo=jokbo)

# 플레이어 추가
player1 = Player("p1")
player1.receive_card([Card(3, is_gwang=True)])
player1.receive_card([Card(7, is_yul=True)])
player2 = Player("p2")
player2.receive_card([Card(4, is_yul=True)])
player2.receive_card([Card(9, is_yul=True)])
player3 = Player("p3")
player3.receive_card([Card(1)])
player3.receive_card([Card(2)])
player4 = Player("p4")
player4.receive_card([Card(1)])
player4.receive_card([Card(2)])

game.add_player(player1)
game.add_player(player2)
game.add_player(player3)
game.add_player(player4)

game.start_game(is_test=True)

({'p1': ['땡잡이', 0], 'p2': ['멍텅구리 구사', 3], 'p3': ['알리', 15], 'p4': ['알리', 15]},
 ({'p1': ['갑오', 9], 'p2': ['끗', 7], 'p3': ['끗', 6], 'p4': ['끗', 7]}, 'p1 우승'))

In [314]:
# TC19

deck = Deck()
game = GameRoom("방1", jokbo=jokbo)

# 플레이어 추가
player1 = Player("p1")
player1.receive_card([Card(3, is_gwang=True)])
player1.receive_card([Card(7, is_yul=True)])
player2 = Player("p2")
player2.receive_card([Card(4, is_yul=True)])
player2.receive_card([Card(9, is_yul=True)])
player3 = Player("p3")
player3.receive_card([Card(4)])
player3.receive_card([Card(9)])
player4 = Player("p4")
player4.receive_card([Card(9)])
player4.receive_card([Card(9)])

game.add_player(player1)
game.add_player(player2)
game.add_player(player3)
game.add_player(player4)

game.start_game(is_test=True)

({'p1': ['땡잡이', 110], 'p2': ['멍텅구리 구사', 3], 'p3': ['구사', 3], 'p4': ['땡', 109]},
 ({'p1': ['끗', 4], 'p2': ['끗', 4], 'p3': ['끗', 6], 'p4': ['갑오', 9]}, 'p4 우승'))

In [315]:
# TC20

deck = Deck()
game = GameRoom("방1", jokbo=jokbo)

# 플레이어 추가
player1 = Player("p1")
player1.receive_card([Card(3, is_gwang=True)])
player1.receive_card([Card(7, is_yul=True)])
player2 = Player("p2")
player2.receive_card([Card(4, is_yul=True)])
player2.receive_card([Card(9, is_yul=True)])
player3 = Player("p3")
player3.receive_card([Card(4)])
player3.receive_card([Card(9)])
player4 = Player("p4")
player4.receive_card([Card(1)])
player4.receive_card([Card(2)])

game.add_player(player1)
game.add_player(player2)
game.add_player(player3)
game.add_player(player4)

game.start_game(is_test=True)

({'p1': ['땡잡이', 0], 'p2': ['멍텅구리 구사', 3], 'p3': ['구사', 3], 'p4': ['알리', 15]},
 ({'p1': ['끗', 6], 'p2': ['갑오', 9], 'p3': ['끗', 5], 'p4': ['장삥', 12]},
  'p4 우승'))

In [316]:
# TC21

deck = Deck()
game = GameRoom("방1", jokbo=jokbo)

# 플레이어 추가
player1 = Player("p1")
player1.receive_card([Card(4, is_yul=True)])
player1.receive_card([Card(9, is_yul=True)])
player2 = Player("p2")
player2.receive_card([Card(9)])
player2.receive_card([Card(9)])
player3 = Player("p3")
player3.receive_card([Card(1)])
player3.receive_card([Card(2)])
player4 = Player("p4")
player4.receive_card([Card(1)])
player4.receive_card([Card(2)])

game.add_player(player1)
game.add_player(player2)
game.add_player(player3)
game.add_player(player4)

game.start_game(is_test=True)

({'p1': ['멍텅구리 구사', 3], 'p2': ['땡', 109], 'p3': ['알리', 15], 'p4': ['알리', 15]},
 ({'p1': ['땡', 101], 'p2': ['끗', 1], 'p3': ['끗', 1], 'p4': ['끗', 8]}, 'p1 우승'))

In [317]:
# TC22

deck = Deck()
game = GameRoom("방1", jokbo=jokbo)

# 플레이어 추가
player1 = Player("p1")
player1.receive_card([Card(4, is_yul=True)])
player1.receive_card([Card(9, is_yul=True)])
player2 = Player("p2")
player2.receive_card([Card(3, is_gwang=True)])
player2.receive_card([Card(8, is_gwang=True)])
player3 = Player("p3")
player3.receive_card([Card(1)])
player3.receive_card([Card(2)])
player4 = Player("p4")
player4.receive_card([Card(1)])
player4.receive_card([Card(2)])

game.add_player(player1)
game.add_player(player2)
game.add_player(player3)
game.add_player(player4)

game.start_game(is_test=True)

({'p1': ['멍텅구리 구사', 3],
  'p2': ['삼팔광땡', 1300],
  'p3': ['알리', 15],
  'p4': ['알리', 15]},
 'p2 우승')

In [318]:
# TC23

deck = Deck()
game = GameRoom("방1", jokbo=jokbo)

# 플레이어 추가
player1 = Player("p1")
player1.receive_card([Card(4, is_yul=True)])
player1.receive_card([Card(9, is_yul=True)])
player2 = Player("p2")
player2.receive_card([Card(4)])
player2.receive_card([Card(9)])
player3 = Player("p3")
player3.receive_card([Card(9)])
player3.receive_card([Card(9)])
player4 = Player("p4")
player4.receive_card([Card(8)])
player4.receive_card([Card(8)])

game.add_player(player1)
game.add_player(player2)
game.add_player(player3)
game.add_player(player4)

game.start_game(is_test=True)

({'p1': ['멍텅구리 구사', 3], 'p2': ['구사', 3], 'p3': ['땡', 109], 'p4': ['땡', 108]},
 ({'p1': ['끗', 3], 'p2': ['끗', 8], 'p3': ['세륙', 10], 'p4': ['끗', 7]}, 'p3 우승'))

In [319]:
# TC24

deck = Deck()
game = GameRoom("방1", jokbo=jokbo)

# 플레이어 추가
player1 = Player("p1")
player1.receive_card([Card(4, is_yul=True)])
player1.receive_card([Card(9, is_yul=True)])
player2 = Player("p2")
player2.receive_card([Card(4)])
player2.receive_card([Card(9)])
player3 = Player("p3")
player3.receive_card([Card(1)])
player3.receive_card([Card(2)])
player4 = Player("p4")
player4.receive_card([Card(1)])
player4.receive_card([Card(2)])

game.add_player(player1)
game.add_player(player2)
game.add_player(player3)
game.add_player(player4)

game.start_game(is_test=True)

({'p1': ['멍텅구리 구사', 3], 'p2': ['구사', 3], 'p3': ['알리', 15], 'p4': ['알리', 15]},
 ({'p1': ['끗', 1], 'p2': ['장삥', 12], 'p3': ['땡', 103], 'p4': ['끗', 8]},
  'p3 우승'))

In [321]:
# TC25

deck = Deck()
game = GameRoom("방1", jokbo=jokbo)

# 플레이어 추가
player1 = Player("p1")
player1.receive_card([Card(4)])
player1.receive_card([Card(9)])
player2 = Player("p2")
player2.receive_card([Card(1)])
player2.receive_card([Card(2)])
player3 = Player("p3")
player3.receive_card([Card(1)])
player3.receive_card([Card(4)])
player4 = Player("p4")
player4.receive_card([Card(1)])
player4.receive_card([Card(9)])

game.add_player(player1)
game.add_player(player2)
game.add_player(player3)
game.add_player(player4)

game.start_game(is_test=True)

({'p1': ['구사', 3], 'p2': ['알리', 15], 'p3': ['독사', 14], 'p4': ['구삥', 13]},
 ({'p1': ['끗', 6], 'p2': ['구삥', 13], 'p3': ['끗', 1], 'p4': ['장사', 11]},
  'p2 우승'))

In [322]:
# TC26

deck = Deck()
game = GameRoom("방1", jokbo=jokbo)

# 플레이어 추가
player1 = Player("p1")
player1.receive_card([Card(4)])
player1.receive_card([Card(9)])
player2 = Player("p2")
player2.receive_card([Card(9)])
player2.receive_card([Card(9)])
player3 = Player("p3")
player3.receive_card([Card(1)])
player3.receive_card([Card(2)])
player4 = Player("p4")
player4.receive_card([Card(1)])
player4.receive_card([Card(4)])

game.add_player(player1)
game.add_player(player2)
game.add_player(player3)
game.add_player(player4)

game.start_game(is_test=True)

({'p1': ['구사', 3], 'p2': ['땡', 109], 'p3': ['알리', 15], 'p4': ['독사', 14]},
 'p2 우승')

In [323]:
# TC27

deck = Deck()
game = GameRoom("방1", jokbo=jokbo)

# 플레이어 추가
player1 = Player("p1")
player1.receive_card([Card(1)])
player1.receive_card([Card(9)])
player2 = Player("p2")
player2.receive_card([Card(1)])
player2.receive_card([Card(10)])
player3 = Player("p3")
player3.receive_card([Card(1)])
player3.receive_card([Card(2)])
player4 = Player("p4")
player4.receive_card([Card(1)])
player4.receive_card([Card(4)])

game.add_player(player1)
game.add_player(player2)
game.add_player(player3)
game.add_player(player4)

game.start_game(is_test=True)

({'p1': ['구삥', 13], 'p2': ['장삥', 12], 'p3': ['알리', 15], 'p4': ['독사', 14]},
 'p3 우승')

In [325]:
# TC28

deck = Deck()
game = GameRoom("방1", jokbo=jokbo)

# 플레이어 추가
player1 = Player("p1")
player1.receive_card([Card(1)])
player1.receive_card([Card(2)])
player2 = Player("p2")
player2.receive_card([Card(1)])
player2.receive_card([Card(2)])
player3 = Player("p3")
player3.receive_card([Card(1)])
player3.receive_card([Card(4)])
player4 = Player("p4")
player4.receive_card([Card(1)])
player4.receive_card([Card(9)])

game.add_player(player1)
game.add_player(player2)
game.add_player(player3)
game.add_player(player4)

game.start_game(is_test=True)

({'p1': ['알리', 15], 'p2': ['알리', 15], 'p3': ['독사', 14], 'p4': ['구삥', 13]},
 ({'p1': ['세륙', 10], 'p2': ['망통', 0]}, 'p1 우승'))